# Imports

In [1]:
from __future__ import annotations

from typing import Dict, List, Optional, Tuple

import numpy as np
import pandas as pd
from sklearn.base import clone
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder


# Dataset preprocess

In [2]:
def add_day_column(df: pd.DataFrame) -> pd.DataFrame:
    # Change 1: use day of year, keep column name "day" to minimize changes elsewhere
    if "day" in df.columns:
        out = df.copy()
        out["day"] = out["day"].astype(int)
        return out

    if "dteday" not in df.columns:
        raise ValueError("column dteday not found, cannot build day")

    out = df.copy()
    d = pd.to_datetime(out["dteday"], errors="raise")
    out["day"] = d.dt.dayofyear.astype(int)
    return out


def drop_target_columns(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for col in ["cnt", "registered"]:
        if col in out.columns:
            out = out.drop(columns=[col])
    return out


def chronological_train_test_split(
    df: pd.DataFrame,
    test_size: float = 0.2,
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    # Change 4: chronological split instead of random split
    out = df.copy()
    if "dteday" in out.columns:
        out["dteday"] = pd.to_datetime(out["dteday"], errors="raise")
        out = out.sort_values("dteday").reset_index(drop=True)
    elif "instant" in out.columns:
        out = out.sort_values("instant").reset_index(drop=True)
    else:
        raise ValueError("need dteday or instant for chronological split")

    n = len(out)
    n_test = int(np.floor(float(test_size) * n))
    n_test = max(1, min(n - 1, n_test))
    n_train = n - n_test

    df_train = out.iloc[:n_train].reset_index(drop=True)
    df_test = out.iloc[n_train:].reset_index(drop=True)
    return df_train, df_test


TEST_SIZE = 0.2

df = pd.read_csv("data/day.csv")
df = add_day_column(df)
df = drop_target_columns(df)

df_train_raw, df_test_raw = chronological_train_test_split(df, test_size=float(TEST_SIZE))

print("full:", df.shape)
print("train:", df_train_raw.shape)
print("test:", df_test_raw.shape)
print(df.head())


FileNotFoundError: [Errno 2] No such file or directory: 'data/day.csv'

# Atemp calculus

In [ ]:
TEMP_SCALE_C = 41.0
ATEMP_SCALE_C = 50.0
WIND_SCALE = 67.0
KPH_TO_MPS = 1.0 / 3.6


def vap_pressure_hpa(temp_c: np.ndarray, hum_frac: np.ndarray) -> np.ndarray:
    t = temp_c.astype(float)
    h = np.clip(hum_frac.astype(float), 0.0, 1.0)
    expo = (17.27 * t) / (237.7 + t)
    es = 6.105 * np.exp(expo)
    return es * h


def atemp_physics_norm_kmph(temp: np.ndarray, hum: np.ndarray, windspeed: np.ndarray) -> np.ndarray:
    temp_c = TEMP_SCALE_C * temp.astype(float)
    hum_frac = np.clip(hum.astype(float), 0.0, 1.0)

    wind_kmph = WIND_SCALE * np.maximum(windspeed.astype(float), 0.0)
    wind_mps = KPH_TO_MPS * wind_kmph

    e_hpa = vap_pressure_hpa(temp_c, hum_frac)
    atemp_c = temp_c + 0.33 * e_hpa - 0.70 * wind_mps - 4.0

    return atemp_c / ATEMP_SCALE_C


def metrics_atemp(y_true_norm: np.ndarray, y_pred_norm: np.ndarray) -> Dict[str, float]:
    mask = np.isfinite(y_true_norm) & np.isfinite(y_pred_norm)
    y = y_true_norm[mask]
    yhat = y_pred_norm[mask]

    rmse_norm = float(np.sqrt(mean_squared_error(y, yhat)))
    mae_norm = float(mean_absolute_error(y, yhat))
    r2 = float(r2_score(y, yhat))
    corr = float(np.corrcoef(y, yhat)[0, 1]) if y.size >= 2 else float("nan")

    y_c = ATEMP_SCALE_C * y
    yhat_c = ATEMP_SCALE_C * yhat
    rmse_c = float(np.sqrt(mean_squared_error(y_c, yhat_c)))
    mae_c = float(mean_absolute_error(y_c, yhat_c))

    err_c = yhat_c - y_c
    abs_err_c = np.abs(err_c)

    return {
        "n": int(y.size),
        "rmse_norm": rmse_norm,
        "mae_norm": mae_norm,
        "r2": r2,
        "corr": corr,
        "rmse_c": rmse_c,
        "mae_c": mae_c,
        "median_err_c": float(np.percentile(err_c, 50)),
        "abs_err_p90_c": float(np.percentile(abs_err_c, 90)),
        "abs_err_p99_c": float(np.percentile(abs_err_c, 99)),
        "pred_min": float(np.min(yhat)) if yhat.size else float("nan"),
        "pred_max": float(np.max(yhat)) if yhat.size else float("nan"),
    }


def print_metrics_block(title: str, m: Dict[str, float]) -> None:
    print("")
    print(title)
    print(f"n: {m['n']}")
    print(f"rmse_norm: {m['rmse_norm']:.6f}")
    print(f"mae_norm: {m['mae_norm']:.6f}")
    print(f"r2: {m['r2']:.6f}")
    print(f"corr: {m['corr']:.6f}")
    print(f"rmse_c: {m['rmse_c']:.6f}")
    print(f"mae_c: {m['mae_c']:.6f}")
    print(f"median_err_c: {m['median_err_c']:.6f}")
    print(f"abs_err_p90_c: {m['abs_err_p90_c']:.6f}")
    print(f"abs_err_p99_c: {m['abs_err_p99_c']:.6f}")
    print(f"pred_min: {m['pred_min']:.6f}")
    print(f"pred_max: {m['pred_max']:.6f}")


def estimate_bias_c_from_train(df_train_for_bias: pd.DataFrame) -> float:
    # Change 2: estimate bias using train split only
    for c in ["temp", "hum", "windspeed", "atemp"]:
        if c not in df_train_for_bias.columns:
            raise ValueError(f"missing column: {c}")

    y_true = df_train_for_bias["atemp"].to_numpy(dtype=float)
    y_pred = atemp_physics_norm_kmph(
        df_train_for_bias["temp"].to_numpy(dtype=float),
        df_train_for_bias["hum"].to_numpy(dtype=float),
        df_train_for_bias["windspeed"].to_numpy(dtype=float),
    )

    m0 = metrics_atemp(y_true, y_pred)
    delta_c = -m0["median_err_c"]
    return float(delta_c)


if "df_train_raw" not in globals():
    raise RuntimeError("df_train_raw is not defined. Run the preprocess cell first.")

bias_c = estimate_bias_c_from_train(df_train_raw)

print(f"bias_c estimated on train only: {bias_c:.6f} C")

# Evaluate on full data using the train estimated bias
y_true_full = df["atemp"].to_numpy(dtype=float)
y_pred_full = atemp_physics_norm_kmph(
    df["temp"].to_numpy(dtype=float),
    df["hum"].to_numpy(dtype=float),
    df["windspeed"].to_numpy(dtype=float),
)

m_full_physics = metrics_atemp(y_true_full, y_pred_full)
print_metrics_block("physics only, km per hour assumption, full data", m_full_physics)

y_pred_full_bias_fixed = y_pred_full + (bias_c / ATEMP_SCALE_C)
m_full_bias = metrics_atemp(y_true_full, y_pred_full_bias_fixed)
print_metrics_block(f"physics plus constant bias fix, full data, added {bias_c:.6f} C", m_full_bias)


So the formula for original atemp is the following

In [ ]:
DEFAULT_ATEMP_K = {
    "TEMP_SCALE_C": 41.0,
    "ATEMP_SCALE_C": 50.0,
    "WIND_SCALE": 67.0,
    "KPH_TO_MPS": 1.0 / 3.6,
    "VAP_A": 6.105,
    "VAP_B": 17.27,
    "VAP_C": 237.7,
    "E_COEFF": 0.33,
    "WIND_COEFF": 0.70,
    "BASE_OFFSET_C": -4.0,
    # Change 2: bias from train only
    "BIAS_C": float(bias_c),
}


In [ ]:
def atemp_from_constants(temp, hum, windspeed, k=DEFAULT_ATEMP_K):
    temp = np.asarray(temp, dtype=float)
    hum = np.clip(np.asarray(hum, dtype=float), 0.0, 1.0)
    windspeed = np.maximum(np.asarray(windspeed, dtype=float), 0.0)

    temp_c = k["TEMP_SCALE_C"] * temp
    e_hpa = k["VAP_A"] * hum * np.exp((k["VAP_B"] * temp_c) / (k["VAP_C"] + temp_c))

    wind_mps = k["WIND_SCALE"] * windspeed * k["KPH_TO_MPS"]

    atemp_c = (
        temp_c
        + k["E_COEFF"] * e_hpa
        - k["WIND_COEFF"] * wind_mps
        + k["BASE_OFFSET_C"]
        + k["BIAS_C"]
    )
    return atemp_c / k["ATEMP_SCALE_C"]


# Shared regression metrics

In [ ]:
# Change 6: single metrics function used everywhere
def reg_metrics(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    mask = np.isfinite(y_true) & np.isfinite(y_pred)
    y = y_true[mask]
    yhat = y_pred[mask]

    rmse = float(np.sqrt(mean_squared_error(y, yhat)))
    mae = float(mean_absolute_error(y, yhat))
    r2 = float(r2_score(y, yhat))
    corr = float(np.corrcoef(y, yhat)[0, 1]) if y.size >= 2 else float("nan")

    return {"n": int(y.size), "rmse": rmse, "mae": mae, "r2": r2, "corr": corr}


def print_reg_metrics(title: str, m: Dict[str, float]) -> None:
    print("")
    print(title)
    print(f"n: {m['n']}")
    print(f"rmse: {m['rmse']:.6f}")
    print(f"mae: {m['mae']:.6f}")
    print(f"r2: {m['r2']:.6f}")
    print(f"corr: {m['corr']:.6f}")


# Generate dataset shift

In [ ]:
def _ohe():
    try:
        return OneHotEncoder(handle_unknown="ignore", sparse_output=False)
    except TypeError:
        return OneHotEncoder(handle_unknown="ignore", sparse=False)


def fit_casual_rf_with_oof(
    df,
    num_cols,
    cat_cols,
    target_col="casual",
    n_splits=5,
    random_state=42,
    n_estimators=2000,
    min_samples_leaf=2,
    log_target=True,
):
    needed = list(num_cols) + list(cat_cols) + [target_col]
    missing = [c for c in needed if c not in df.columns]
    if missing:
        raise ValueError(f"missing columns for casual model: {missing}")

    df2 = df[needed].copy().dropna(axis=0).reset_index(drop=True)
    x = df2[num_cols + cat_cols].copy()
    y = df2[target_col].to_numpy(dtype=float)

    if log_target:
        y_train = np.log1p(np.maximum(y, 0.0))
    else:
        y_train = y

    pre = ColumnTransformer(
        transformers=[
            ("num", "passthrough", num_cols),
            ("cat", _ohe(), cat_cols),
        ],
        remainder="drop",
    )

    rf = RandomForestRegressor(
        n_estimators=int(n_estimators),
        max_depth=None,
        min_samples_leaf=int(min_samples_leaf),
        n_jobs=1,
        random_state=int(random_state),
    )

    pipe_template = Pipeline([("pre", pre), ("rf", rf)])

    kf = KFold(n_splits=int(n_splits), shuffle=True, random_state=int(random_state))
    yhat_oof = np.full(y.shape[0], np.nan, dtype=float)

    for i, (tr, te) in enumerate(kf.split(x), start=1):
        print(f"[debug] fold {i} of {n_splits}, train={len(tr)}, test={len(te)}")
        pipe_i = clone(pipe_template)
        pipe_i.fit(x.iloc[tr], y_train[tr])

        pred = pipe_i.predict(x.iloc[te]).astype(float)
        if log_target:
            pred = np.expm1(pred)
        pred = np.maximum(pred, 0.0)
        yhat_oof[te] = pred

    m = reg_metrics(y, yhat_oof)
    print("")
    print("CASUAL OOF METRICS")
    print(f"n: {m['n']}")
    print(f"rmse: {m['rmse']:.6f}")
    print(f"mae: {m['mae']:.6f}")
    print(f"r2: {m['r2']:.6f}")
    print(f"corr: {m['corr']:.6f}")

    pipe_final = clone(pipe_template)
    pipe_final.fit(x, y_train)

    print("")
    print("CASUAL FINAL FIT DONE")
    print(f"rows used: {len(df2)}")
    print(f"log_target: {log_target}")

    return pipe_final, log_target, m


def predict_casual(pipe, df_features, log_target=True):
    pred = pipe.predict(df_features).astype(float)
    if log_target:
        pred = np.expm1(pred)
    return np.maximum(pred, 0.0)


In [ ]:
print("============================================================")
print("PART 1, FIT RANDOM FOREST FOR CASUAL (TRAIN SPLIT ONLY) AND SHOW OOF METRICS")
print("============================================================")

if "df_train_raw" not in globals():
    raise RuntimeError("df_train_raw is not defined. Run the preprocess cell first.")

num_cols = ["day", "atemp"]
cat_cols = ["mnth", "yr", "weathersit", "weekday", "holiday", "workingday"]

casual_pipe, casual_log_target, casual_oof_metrics = fit_casual_rf_with_oof(
    df=df_train_raw,
    num_cols=num_cols,
    cat_cols=cat_cols,
    target_col="casual",
    n_splits=5,
    random_state=42,
    n_estimators=2000,
    min_samples_leaf=2,
    log_target=True,
)

print("")
print("============================================================")
print("PART 2, GENERATE SHIFTED DATASET (TEST SPLIT ONLY)")
print("Intervention: make temp less important, increase wind and humidity effects")
print("Then sample from a Negative Binomial to add overdispersion")
print("============================================================")

if "df_test_raw" not in globals():
    raise RuntimeError("df_test_raw is not defined. Run the preprocess cell first.")

k_shift = dict(DEFAULT_ATEMP_K)
k_shift["TEMP_SCALE_C"] = 25.0
k_shift["E_COEFF"] = 0.70
k_shift["VAP_A"] = 8.0
k_shift["WIND_COEFF"] = 1.60

df_test_shifted = df_test_raw.copy()

df_test_shifted["atemp"] = atemp_from_constants(
    df_test_shifted["temp"].to_numpy(dtype=float),
    df_test_shifted["hum"].to_numpy(dtype=float),
    df_test_shifted["windspeed"].to_numpy(dtype=float),
    k=k_shift,
)

df_test_shifted["atemp"] = np.clip(df_test_shifted["atemp"].to_numpy(dtype=float), 0.0, 1.0)

x_shift = df_test_shifted[num_cols + cat_cols].copy()
casual_mean = predict_casual(casual_pipe, x_shift, log_target=casual_log_target)

# Change 3: mean preserving overdispersed counts via Negative Binomial
NB_RANDOM_STATE = 42
NB_SIZE = 20.0  # larger means closer to Poisson
rng = np.random.default_rng(int(NB_RANDOM_STATE))

mu = np.clip(np.asarray(casual_mean, dtype=float), 0.0, None)
size = float(NB_SIZE)
p = size / (size + mu)
p = np.clip(p, 1e-12, 1.0)

casual_noisy = rng.negative_binomial(n=size, p=p).astype(float)

df_test_shifted["casual_mean"] = casual_mean
df_test_shifted["casual"] = casual_noisy

print("SHIFT DONE (test only, Negative Binomial noise)")
print(f"rows: {len(df_test_shifted)}")
print(f"atemp min: {float(df_test_shifted['atemp'].min()):.6f}")
print(f"atemp max: {float(df_test_shifted['atemp'].max()):.6f}")
print(f"casual_mean min: {float(df_test_shifted['casual_mean'].min()):.6f}")
print(f"casual_mean max: {float(df_test_shifted['casual_mean'].max()):.6f}")
print(f"casual (noisy) min: {float(df_test_shifted['casual'].min()):.6f}")
print(f"casual (noisy) max: {float(df_test_shifted['casual'].max()):.6f}")
print(df_test_shifted.head())


# Experiment

In [ ]:
TARGET = "casual"


def _pick_col(df: pd.DataFrame, candidates: list[str]) -> str:
    for c in candidates:
        if c in df.columns:
            return c
    raise ValueError(f"none of these columns exist: {candidates}")


def prep_df(df: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
    missing = [c for c in cols if c not in df.columns]
    if missing:
        raise ValueError(f"missing columns: {missing}")
    return df[cols].copy().dropna(axis=0).reset_index(drop=True)


def fit_rf_regressor(
    df_train: pd.DataFrame,
    feature_cols: list[str],
    cat_cols: list[str],
    num_cols: list[str],
    target_col: str,
    random_state: int = 42,
    n_estimators: int = 2000,
    min_samples_leaf: int = 2,
    log_target: bool = True,
):
    x = df_train[feature_cols].copy()
    y = df_train[target_col].to_numpy(dtype=float)

    if log_target:
        y_fit = np.log1p(np.maximum(y, 0.0))
    else:
        y_fit = y

    pre = ColumnTransformer(
        transformers=[
            ("num", "passthrough", num_cols),
            ("cat", _ohe(), cat_cols),
        ],
        remainder="drop",
    )

    rf = RandomForestRegressor(
        n_estimators=int(n_estimators),
        max_depth=None,
        min_samples_leaf=int(min_samples_leaf),
        n_jobs=-1,
        random_state=int(random_state),
    )

    pipe = Pipeline([("pre", pre), ("rf", rf)])
    pipe.fit(x, y_fit)

    return pipe, bool(log_target)


def predict_rf(pipe: Pipeline, df_in: pd.DataFrame, feature_cols: list[str], log_target: bool) -> np.ndarray:
    pred = pipe.predict(df_in[feature_cols]).astype(float)
    if log_target:
        pred = np.expm1(pred)
    return np.maximum(pred, 0.0)


def eval_on_split(name: str, df_split: pd.DataFrame, direct_pack, lvl2_pack) -> None:
    direct_pipe, direct_log, direct_feat = direct_pack
    lvl2_pipe, lvl2_log, lvl2_feat = lvl2_pack

    y_true = df_split[TARGET].to_numpy(dtype=float)

    yhat_direct = predict_rf(direct_pipe, df_split, direct_feat, direct_log)
    print_reg_metrics(f"{name} direct RF", reg_metrics(y_true, yhat_direct))

    yhat_lvl2 = predict_rf(lvl2_pipe, df_split, lvl2_feat, lvl2_log)
    print_reg_metrics(f"{name} level 2 stable estimator", reg_metrics(y_true, yhat_lvl2))


In [ ]:
print("============================================================")
print("PART 1, TRAIN 2 MODELS ON TRAIN SPLIT, THEN EVALUATE")
print("============================================================")

if "df_train_raw" not in globals() or "df_test_raw" not in globals():
    raise RuntimeError("df_train_raw or df_test_raw not defined. Run the preprocess cell first.")

col_day = _pick_col(df, ["day", "doy", "dayofyear"])
col_month = _pick_col(df, ["month", "mnth"])
col_season = _pick_col(df, ["season"])
col_hum = _pick_col(df, ["humidity", "hum"])
col_temp = _pick_col(df, ["temp"])
col_weathersit = _pick_col(df, ["weathersit"])
col_wind = _pick_col(df, ["windspeed"])
col_atemp = _pick_col(df, ["atemp"])
col_holiday = _pick_col(df, ["holiday"])
col_weekday = _pick_col(df, ["weekday"])
col_workingday = _pick_col(df, ["workingday"])
col_year = _pick_col(df, ["year", "yr"])

all_features_direct = [
    col_day, col_month, col_year, col_season, col_hum, col_temp, col_weathersit, col_wind, col_atemp,
    col_holiday, col_weekday, col_workingday
]

z_features = [col_day, col_month, col_year, col_weathersit, col_weekday, col_workingday, col_holiday]
features_lvl2 = z_features + [col_atemp]

cat_cols_all = [col_month, col_year, col_season, col_weathersit, col_weekday, col_holiday, col_workingday]

num_cols_direct = [c for c in all_features_direct if c not in cat_cols_all]
num_cols_lvl2 = [c for c in features_lvl2 if c not in cat_cols_all]

needed_cols = sorted(set(all_features_direct + [TARGET]))
df_train = prep_df(df_train_raw, needed_cols)
df_test = prep_df(df_test_raw, needed_cols)

direct_pipe, direct_log = fit_rf_regressor(
    df_train=df_train,
    feature_cols=all_features_direct,
    cat_cols=[c for c in cat_cols_all if c in all_features_direct],
    num_cols=num_cols_direct,
    target_col=TARGET,
    random_state=42,
    n_estimators=2000,
    min_samples_leaf=2,
    log_target=True,
)

lvl2_pipe, lvl2_log = fit_rf_regressor(
    df_train=df_train,
    feature_cols=features_lvl2,
    cat_cols=[c for c in cat_cols_all if c in features_lvl2],
    num_cols=num_cols_lvl2,
    target_col=TARGET,
    random_state=42,
    n_estimators=2000,
    min_samples_leaf=2,
    log_target=True,
)

direct_pack = (direct_pipe, direct_log, all_features_direct)
lvl2_pack = (lvl2_pipe, lvl2_log, features_lvl2)

eval_on_split("train", df_train, direct_pack, lvl2_pack)
eval_on_split("test", df_test, direct_pack, lvl2_pack)


In [ ]:
print("============================================================")
print("PART 2, EVALUATE BOTH MODELS ON SHIFTED TEST SPLIT")
print("============================================================")

if "df_test_shifted" in globals():
    neededS = sorted(set(all_features_direct + [TARGET]))
    dfS = prep_df(df_test_shifted, neededS)

    yS_true = dfS[TARGET].to_numpy(dtype=float)

    yS_direct = predict_rf(direct_pipe, dfS, all_features_direct, direct_log)
    print_reg_metrics("shifted test direct RF", reg_metrics(yS_true, yS_direct))

    yS_lvl2 = predict_rf(lvl2_pipe, dfS, features_lvl2, lvl2_log)
    print_reg_metrics("shifted test level 2 stable estimator", reg_metrics(yS_true, yS_lvl2))

    diffS = np.subtract(yS_lvl2, yS_direct)

    print("")
    print("shifted test comparison (level2 minus direct)")
    print(f"mean: {float(np.mean(diffS)):.6f}")
    print(f"p50: {float(np.percentile(diffS, 50)):.6f}")
    print(f"p90(abs): {float(np.percentile(np.abs(diffS), 90)):.6f}")
    print(f"p99(abs): {float(np.percentile(np.abs(diffS), 99)):.6f}")
else:
    print("df_test_shifted is not defined in this notebook session, run the shift cell first")


# Shift diagnostics plots

These plots compare original test versus shifted test to verify that only the intended mechanism moved, and to inspect how labels relate to atemp.


In [ ]:
import matplotlib.pyplot as plt

if "df_test_raw" not in globals() or "df_test_shifted" not in globals():
    raise RuntimeError("df_test_raw or df_test_shifted not found. Run the earlier cells first.")

# Helper to keep code short

def _as_np(s):
    return s.to_numpy(dtype=float)

# 1) Distributions of atemp, original test vs shifted test
plt.figure(figsize=(9, 5))
plt.hist(_as_np(df_test_raw["atemp"]), bins=30, alpha=0.5, label="test original")
plt.hist(_as_np(df_test_shifted["atemp"]), bins=30, alpha=0.5, label="test shifted")
plt.title("atemp distribution")
plt.xlabel("atemp")
plt.ylabel("count")
plt.legend()
plt.grid(True, alpha=0.2)
plt.show()

# 2) Confirm temp, hum, windspeed distributions did not change
for col in ["temp", "hum", "windspeed"]:
    plt.figure(figsize=(9, 5))
    plt.hist(_as_np(df_test_raw[col]), bins=30, alpha=0.5, label=f"test original {col}")
    plt.hist(_as_np(df_test_shifted[col]), bins=30, alpha=0.5, label=f"test shifted {col}")
    plt.title(f"{col} distribution")
    plt.xlabel(col)
    plt.ylabel("count")
    plt.legend()
    plt.grid(True, alpha=0.2)
    plt.show()

# 3) Distributions of casual, original test vs shifted test
plt.figure(figsize=(9, 5))
plt.hist(_as_np(df_test_raw["casual"]), bins=30, alpha=0.5, label="test original casual")
plt.hist(_as_np(df_test_shifted["casual"]), bins=30, alpha=0.5, label="test shifted casual")
plt.title("casual distribution")
plt.xlabel("casual")
plt.ylabel("count")
plt.legend()
plt.grid(True, alpha=0.2)
plt.show()

# 4) Scatter: casual vs atemp, original test and shifted test
plt.figure(figsize=(9, 5))
plt.scatter(_as_np(df_test_raw["atemp"]), _as_np(df_test_raw["casual"]), s=20, alpha=0.35, label="test original")
plt.scatter(_as_np(df_test_shifted["atemp"]), _as_np(df_test_shifted["casual"]), s=20, alpha=0.35, label="test shifted")
plt.title("casual vs atemp")
plt.xlabel("atemp")
plt.ylabel("casual")
plt.legend()
plt.grid(True, alpha=0.2)
plt.show()

# 5) Residuals: observed atemp minus physics atemp under the original mechanism
# This should shift if you changed the mechanism that generates atemp.

atemp_phy_test = atemp_from_constants(
    _as_np(df_test_raw["temp"]),
    _as_np(df_test_raw["hum"]),
    _as_np(df_test_raw["windspeed"]),
    k=DEFAULT_ATEMP_K,
)

res_test = _as_np(df_test_raw["atemp"]) - atemp_phy_test
res_shift = _as_np(df_test_shifted["atemp"]) - atemp_phy_test

plt.figure(figsize=(9, 5))
plt.hist(res_test, bins=30, alpha=0.5, label="test original residual")
plt.hist(res_shift, bins=30, alpha=0.5, label="test shifted residual")
plt.title("atemp residuals relative to original physics")
plt.xlabel("atemp observed minus atemp physics")
plt.ylabel("count")
plt.legend()
plt.grid(True, alpha=0.2)
plt.show()

# 6) Optional, conditional means: E[casual | atemp bin] for original and shifted
# Uses shared bins to make curves comparable.

bins = np.linspace(0.0, 1.0, 11)
centers = 0.5 * (bins[:-1] + bins[1:])

b_test = pd.cut(df_test_raw["atemp"], bins=bins, include_lowest=True)
b_shift = pd.cut(df_test_shifted["atemp"], bins=bins, include_lowest=True)

m_test = df_test_raw.groupby(b_test)["casual"].mean().to_numpy(dtype=float)
m_shift = df_test_shifted.groupby(b_shift)["casual"].mean().to_numpy(dtype=float)

plt.figure(figsize=(9, 5))
plt.plot(centers, m_test, marker="o", label="test original")
plt.plot(centers, m_shift, marker="o", label="test shifted")
plt.title("mean casual by atemp bin")
plt.xlabel("atemp")
plt.ylabel("mean casual")
plt.legend()
plt.grid(True, alpha=0.2)
plt.show()